In [5]:
!pip install selenium

In [19]:
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
import pandas as pd
from urllib.request import urlopen
from urllib.parse import quote
import time
import re

# 위치 지정
driver = webdriver.Chrome("d:/py_data/chromedriver.exe")
keys = Keys()

driver.get("https://section.cafe.naver.com/")

keyword = input("찾으시려는 키워드를 입력하세요:")
end_page = int(input("마지막 페이지를 입력하세요: ")) + 1
page_url = "&p="

query_elm = driver.find_element_by_css_selector("#header > div.snb_area > div > form > fieldset > div > div > input")
query_elm.click() # 검색창 클릭
query_elm.send_keys(keyword) #검색어 입력

# 엔터키 입력
query_elm.send_keys(keys.ENTER)
driver.set_page_load_timeout(10) # 로드 대기시간
driver.implicitly_wait(10)  # 브라우저에서 사용되는 엔진 자체에서 파싱되는 시간을 기다려 주는 메소드
#time.sleep(2)     # 프로세스 자체를 지정한 시간동안 기다려 주는 역할(무조건 대기)

# 전체글 클릭
driver.find_element_by_css_selector("#app > div > div.container > div > div.SectionSearchContent > div.section_search_tab_list > a:nth-child(3)").click()
url = driver.current_url
driver.implicitly_wait(10) 

title_lst = []
doc_lst = []
comment_lst = []

for now_page in range(1,end_page):
    driver.get(url+page_url+str(now_page))
    time.sleep(2)     # 프로세스 자체를 지정한 시간동안 기다려 주는 역할(무조건 대기)
    html = driver.page_source
    soup = bs(html, 'html.parser')
    item_lst = soup.find_all("div",class_="item_inner")
    
    for i in range(1,len(item_lst)+1):
        driver.find_element_by_css_selector("#app > div > div.container > div > div.SectionSearchContent > div.section_search_content > div > div.article_list_area > ul > li:nth-child("+str(i)+") > div > div > div > a").click()
        driver.implicitly_wait(10) 
        driver.switch_to.window(driver.window_handles[-1])
        driver.switch_to_frame('cafe_main')
        driver.implicitly_wait(40) 
        
        html = driver.page_source
        soup = bs(html, 'html.parser')
        #print(soup.find("h3",class_="title_text"))
        
             
        try:     
            title = re.sub('[^a-zA-Zㄱ-ㅣ가-힣0-9 ]',"",soup.find('h3',class_='title_text').text)  
        except:
            title=" "
            
        title_lst.append(title)
        
        try:
            doc_temp = re.sub('[^a-zA-Zㄱ-ㅣ가-힣0-9 ]',"",soup.find("div",class_="se-module").text)
        except:
            doc_temp = " "
        doc_lst.append(doc_temp)
        
        comment_area = soup.find_all("div", class_="comment_area")
        comment = ""
        
        for i in comment_area:
            try:
                comment_temp = i.find("span",class_="text_comment").text
            except AttributeError:
                pass
            comment = comment + re.sub('[^a-zA-Zㄱ-ㅣ가-힣 ]', '',comment_temp)
        comment_lst.append(comment)
        
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        driver.implicitly_wait(10) 

driver.close()

df = pd.DataFrame({"title":title_lst, "doc":doc_lst, "comment":comment_lst})
display(df)
df.to_csv("./data/"+keyword+" 네이버 카페 크롤링.csv",encoding="cp949")

찾으시려는 키워드를 입력하세요:국내여행
마지막 페이지를 입력하세요: 2


<ipython-input-19-0b1a8c85e923>:51: DeprecationWarning: use driver.switch_to.frame instead
  driver.switch_to_frame('cafe_main')


,title,doc,comment
0,아고다 2월 할인코드 국내여행지 적용,아고다 2월 할인코드 국내여행지 적용,이렇게 미리 알고 아고다 월 할인코드 이용해서 호텔 예약하면 되겠네여여기서 확실하게...
1,,,
2,트립닷컴 1월 할인코드 국내여행지 사용,트립닷컴 1월 할인코드 국내여행지 사용,여러므로 트립닷컴 월 할인코드 적용이 간편해지는것 같아서 다행입니당그래서 트립닷컴월...
3,국내여행 캠핑하러,국내여행 캠핑하러 여러분 한 여름에 유일에 제가 좋아하는것은은하수입니다 매일 덥지만...,여기서 모든걸 편하게 알아가시고 쿠폰만 적용하시면 할인 받으실 수 있습니다영양제 구...
4,국내여행지추천 태안 즐길거리,,와 힐링 하고 오셨겠어요재밌게 놀고 오셨네요 ㅎㅎ저도 올해는 서해바다로 한번 가...
5,국내여행 부산거제남해,처음엔 제주도로 갈까생각했지만 제주도는 날따뜻해지면 가기로 하고 남해쪽으로 가기로 ...,부산 사는데 돼지국밥 꼭 먹고가세요 ㅎㅎ남해가 진짜 좋다고 하더라구요아래는 날씨 따...
6,,,
7,,,
8,코로나 국내여행 해안도로차박,코로나로 집에만 있으니 답답하죠코로나 국내여행 방법에 대해 소개하려고 해요요즘은 사...,코로나로 인해 강제 드라이브 국내여행인 세상이 되었네요 ㅠㅠ이 기회에 국내도 구석구...
9,베트남 국내 여행 중 여권 만료 기간,해외 입출국시에는 여권 만료기간이 6개월 이상 남아있어야 하는 걸로 알고 있어요베트...,거주증이 있으면 상관없는걸로 알고있어요 그래도모르니 영사관이나 여행사 문의한번해보세...


In [22]:
comment_txt=""

for i in range(len(df)):
    comment_txt=comment_txt+str(df['comment'].loc[i])
for txt in df['comment']:
    comment_txt=comment_txt+str(txt)
comment_txt

'이렇게 미리 알고 아고다 월 할인코드 이용해서 호텔 예약하면 되겠네여여기서 확실하게 호텔 예약하고 호캉스 준비할 수 있을 것 같습니다 ㅎㅎ저도 이번에 아고다 통해서 할인코드 이용하고 월 여행 준비해보려구요앞으로도 좋은 소식들 많이 전해주시면 아고다 월 할인코드 꼭 사용할게여역시 아고다가 제일 저렴하네요 감사해요조금더 편하게 여행준비가 가능해질거 같습니다ㅎㅎ가까운곳으로 가볍게 호캉스 다녀오고 싶긴 해용여기저기 힘들게 다니면서 쿠폰찾을 필요 없겠어용확실히 예전보다 할인되는 부분이 많은듯 해여아고다 월 할인코드 사용이 너무 필요해보입니당저는 서울신라호텔 다녀올까 고민중입니다ㅋㅋ미리 알아두고 편하게 사용해보도록 할게여ㅋㅋ국내여행지로 가볍게 호캉스 다녀올까 고민중입니당여러므로 트립닷컴 월 할인코드 적용이 간편해지는것 같아서 다행입니당그래서 트립닷컴월할인코드사용이필요하죠모든분들이 편하게 여행하실 수 있는 날이 올까요ㅠㅠ항상좋은정보 알려주셔서 감사드려용다들 여기서 편하게 예약하실 수 있다니 너무 좋은것 같습니다 ㅎㅎ빠르게 쓰는 트립닷컴 월 할인코드라서 편하고 좋네유앞으로도 트립닷컴 월 할인코드 같이 꼭 필요한 내용들 많이 알려드릴게용이번에도 호캉스준비하기 넘 편하고 좋겠네요ㅋㅋ호캉스 준비하시기에는 이만한 쿠폰이 없지 않을까 생각됩니다여기서 미리 알아두니깐 편하고 너무 좋았습니당미리 알아두니깐 확실히 간편해서 좋습니다ㅋㅋ꾸준히 좋은정보들 많이 알려주시길 기다리고 있어여트립닷컴 월 할인코드 사용이 편해요ㅎㅎ여기서 모든걸 편하게 알아가시고 쿠폰만 적용하시면 할인 받으실 수 있습니다영양제 구입할때 만족스럽네요저도 고마워요  저도 정말 고마워요  굳이 다른데서 힘들게 찾아볼 필요가 없겠습니당ㅎ에서 건강관련 제품 주문할 때 할인코드 챙기는건 필수죠  사람들이 에서 구매하는 이유가 이런거네요 저만 몰랐네요 비가 많이 오는 요즘 참고하면 도움될만한 소식이라고 생각함코로나가 하루 빨리 사라지길 기다리고 있어요ㅠ영양제 구입할때 유용해요그래서 원래 좀 비싸도 부담이 적어져요  와 힐링 하고

In [23]:
from konlpy.tag import Okt

okt=Okt()
명사=okt.nouns(comment_txt)
명사

['미리',
 '알',
 '아고다',
 '월',
 '할인',
 '코드',
 '이용',
 '호텔',
 '예약',
 '여기',
 '호텔',
 '예약',
 '호캉스',
 '준비',
 '수',
 '것',
 '저',
 '이번',
 '아고다',
 '통해',
 '할인',
 '코드',
 '이용',
 '월',
 '여행',
 '준비',
 '앞',
 '소식',
 '전',
 '아고다',
 '월',
 '할인',
 '코드',
 '꼭',
 '사용',
 '역시',
 '아고다',
 '제일',
 '조금',
 '더',
 '여행',
 '준비',
 '거',
 '곳',
 '호캉스',
 '여기저기',
 '쿠폰',
 '필요',
 '예전',
 '할인',
 '부분',
 '해',
 '아고다',
 '월',
 '할인',
 '코드',
 '사용',
 '저',
 '서울',
 '신라',
 '호텔',
 '고민',
 '미리',
 '알',
 '아두',
 '사용',
 '게',
 '국내',
 '여행지',
 '호캉스',
 '고민',
 '므',
 '트립',
 '월',
 '할인',
 '코드',
 '적용',
 '다행',
 '트립',
 '월',
 '할인',
 '코드',
 '사용',
 '모든',
 '여행',
 '수',
 '날',
 '요',
 '항상',
 '정보',
 '감사',
 '여기',
 '예약',
 '수',
 '것',
 '트립',
 '월',
 '할인',
 '코드',
 '유앞',
 '트립',
 '월',
 '할인',
 '코드',
 '꼭',
 '내용',
 '드릴',
 '용',
 '이번',
 '호',
 '캉스',
 '준비',
 '호캉스',
 '준비',
 '시기',
 '이만',
 '쿠폰',
 '생각',
 '여기',
 '미리',
 '알',
 '아두',
 '습',
 '미리',
 '알',
 '아두',
 '정보',
 '트립',
 '월',
 '할인',
 '코드',
 '사용',
 '여기',
 '걸',
 '알',
 '아가시',
 '쿠폰',
 '적용',
 '할인',
 '수',
 '양제',
 '구입',
 '때',
 '도',
 '저',
 '정말',
 '굳이',

In [27]:
from collections import Counter
count=Counter(명사)
count_rank=count.most_common()
count_rank

[('여행', 56),
 ('국내', 40),
 ('부산', 36),
 ('저', 32),
 ('곳', 26),
 ('할인', 24),
 ('때', 24),
 ('도', 22),
 ('코드', 20),
 ('정말', 20),
 ('코로나', 20),
 ('월', 18),
 ('한번', 18),
 ('통영', 18),
 ('전', 16),
 ('좀', 16),
 ('진짜', 16),
 ('여수', 16),
 ('수', 14),
 ('꼭', 14),
 ('더', 14),
 ('사진', 14),
 ('여기', 12),
 ('것', 12),
 ('정보', 12),
 ('날씨', 12),
 ('요전', 12),
 ('호주', 12),
 ('순천', 12),
 ('알', 10),
 ('아고다', 10),
 ('준비', 10),
 ('사용', 10),
 ('트립', 10),
 ('용', 10),
 ('생각', 10),
 ('남해', 10),
 ('경주', 10),
 ('속초', 10),
 ('해외여행', 10),
 ('온천', 10),
 ('미리', 8),
 ('호텔', 8),
 ('호캉스', 8),
 ('이번', 8),
 ('습', 8),
 ('힐링', 8),
 ('지금', 8),
 ('시간', 8),
 ('아이', 8),
 ('또', 8),
 ('울릉도', 8),
 ('그냥', 8),
 ('고성', 8),
 ('거제', 8),
 ('매니아', 8),
 ('풍경', 8),
 ('일본', 8),
 ('구례', 8),
 ('예약', 6),
 ('앞', 6),
 ('쿠폰', 6),
 ('해', 6),
 ('고민', 6),
 ('아두', 6),
 ('항상', 6),
 ('사람', 6),
 ('비', 6),
 ('거제도', 6),
 ('기회', 6),
 ('거주', 6),
 ('문의', 6),
 ('비자', 6),
 ('제', 6),
 ('추천', 6),
 ('울산', 6),
 ('춘천', 6),
 ('겨울', 6),
 ('강원도', 6),
 ('가을', 6),
 ('전주', 6

In [45]:
from ckonlpy.tag import Twitter
from ckonlpy.tag import Postprocessor
Twitter=Twitter()

Twitter.add_dictionary('할인코드','Noun')


In [44]:
명사=Twitter.nouns(comment_txt)
명사

['미리',
 '알',
 '아고다',
 '월',
 '할인코드',
 '이용',
 '호텔',
 '예약',
 '여기',
 '확실',
 '호텔',
 '예약',
 '호캉스',
 '준비',
 '수',
 '것',
 '저',
 '이번',
 '아고다',
 '통해',
 '할인코드',
 '이용',
 '월',
 '여행',
 '준비',
 '앞',
 '소식',
 '전',
 '해주시',
 '아고다',
 '월',
 '할인코드',
 '꼭',
 '사용',
 '게',
 '역시',
 '아고다',
 '제일',
 '저렴',
 '감',
 '사해',
 '요조',
 '금더',
 '여행',
 '준비',
 '가능',
 '까운',
 '곳',
 '호캉스',
 '여기저기',
 '면서',
 '쿠폰',
 '필요',
 '확실',
 '예전',
 '할인',
 '부분',
 '해',
 '아고다',
 '월',
 '할인코드',
 '사용',
 '필요',
 '저',
 '서울',
 '신라호',
 '텔',
 '고민',
 '중',
 '미리',
 '알',
 '아두',
 '사',
 '용해',
 '보도',
 '록',
 '게',
 '국내',
 '여행지',
 '호캉스',
 '고민',
 '중',
 '여러',
 '트립',
 '월',
 '할인코드',
 '적용',
 '간편',
 '해지',
 '것',
 '다행',
 '래서',
 '트립',
 '월',
 '할인코드',
 '사용',
 '필',
 '요하',
 '모든',
 '분',
 '여행',
 '수',
 '날',
 '요',
 '항상',
 '은정',
 '보',
 '감사',
 '여기',
 '예약',
 '수',
 '것',
 '트립',
 '월',
 '할인코드',
 '라서',
 '편',
 '유앞',
 '트립',
 '월',
 '할인코드',
 '꼭',
 '필요',
 '내용',
 '드릴',
 '용',
 '이번',
 '호캉스',
 '준비',
 '편',
 '호캉스',
 '준비',
 '시기',
 '쿠폰',
 '생각',
 '여기',
 '미리',
 '알',
 '아두',
 '편',
 '습',
 '미리',
 '알',
 '아두',
 '확실'

In [42]:
passwords={'제주','제주도','속초','고성','부산','여수','남해','담양','경주'}
postprocessor=Postprocessor(Twitter,passwords=passwords)
postprocessor.pos(comment_txt)

[('부산', 'Noun'),
 ('남해', 'Noun'),
 ('부산', 'Noun'),
 ('부산', 'Noun'),
 ('남해', 'Noun'),
 ('남해', 'Noun'),
 ('부산', 'Noun'),
 ('부산', 'Noun'),
 ('경주', 'Noun'),
 ('부산', 'Noun'),
 ('부산', 'Noun'),
 ('여수', 'Noun'),
 ('남해', 'Noun'),
 ('부산', 'Noun'),
 ('경주', 'Noun'),
 ('부산', 'Noun'),
 ('부산', 'Noun'),
 ('속초', 'Noun'),
 ('고성', 'Noun'),
 ('속초', 'Noun'),
 ('부산', 'Noun'),
 ('경주', 'Noun'),
 ('고성', 'Noun'),
 ('남해', 'Noun'),
 ('제주', 'Noun'),
 ('부산', 'Noun'),
 ('경주', 'Noun'),
 ('속초', 'Noun'),
 ('여수', 'Noun'),
 ('속초', 'Noun'),
 ('여수', 'Noun'),
 ('여수', 'Noun'),
 ('부산', 'Noun'),
 ('부산', 'Noun'),
 ('부산', 'Noun'),
 ('제주도', 'Noun'),
 ('부산', 'Noun'),
 ('경주', 'Noun'),
 ('속초', 'Noun'),
 ('여수', 'Noun'),
 ('여수', 'Noun'),
 ('여수', 'Noun'),
 ('제주도', 'Noun'),
 ('부산', 'Noun'),
 ('여수', 'Noun'),
 ('제주도', 'Noun'),
 ('부산', 'Noun'),
 ('부산', 'Noun'),
 ('남해', 'Noun'),
 ('부산', 'Noun'),
 ('부산', 'Noun'),
 ('남해', 'Noun'),
 ('남해', 'Noun'),
 ('부산', 'Noun'),
 ('부산', 'Noun'),
 ('경주', 'Noun'),
 ('부산', 'Noun'),
 ('부산', 'Noun'),
 ('여수', 'No

In [46]:
dict(count_rank)

{'여행': 56,
 '국내': 40,
 '부산': 36,
 '저': 32,
 '곳': 26,
 '할인': 24,
 '때': 24,
 '도': 22,
 '코드': 20,
 '정말': 20,
 '코로나': 20,
 '월': 18,
 '한번': 18,
 '통영': 18,
 '전': 16,
 '좀': 16,
 '진짜': 16,
 '여수': 16,
 '수': 14,
 '꼭': 14,
 '더': 14,
 '사진': 14,
 '여기': 12,
 '것': 12,
 '정보': 12,
 '날씨': 12,
 '요전': 12,
 '호주': 12,
 '순천': 12,
 '알': 10,
 '아고다': 10,
 '준비': 10,
 '사용': 10,
 '트립': 10,
 '용': 10,
 '생각': 10,
 '남해': 10,
 '경주': 10,
 '속초': 10,
 '해외여행': 10,
 '온천': 10,
 '미리': 8,
 '호텔': 8,
 '호캉스': 8,
 '이번': 8,
 '습': 8,
 '힐링': 8,
 '지금': 8,
 '시간': 8,
 '아이': 8,
 '또': 8,
 '울릉도': 8,
 '그냥': 8,
 '고성': 8,
 '거제': 8,
 '매니아': 8,
 '풍경': 8,
 '일본': 8,
 '구례': 8,
 '예약': 6,
 '앞': 6,
 '쿠폰': 6,
 '해': 6,
 '고민': 6,
 '아두': 6,
 '항상': 6,
 '사람': 6,
 '비': 6,
 '거제도': 6,
 '기회': 6,
 '거주': 6,
 '문의': 6,
 '비자': 6,
 '제': 6,
 '추천': 6,
 '울산': 6,
 '춘천': 6,
 '겨울': 6,
 '강원도': 6,
 '가을': 6,
 '전주': 6,
 '어디': 6,
 '년': 6,
 '정원': 6,
 '홍학': 6,
 '제주도': 6,
 '매력': 6,
 '마을': 6,
 '쪽': 6,
 '이용': 4,
 '소식': 4,
 '제일': 4,
 '필요': 4,
 '게': 4,
 '여행지': 4,
 '적용': 4,
 '날': 4,
 

In [51]:
from wordcloud import WordCloud, STOPWORDS#필요없는 데이터 제거
import nltk

ko_txt=nltk.Text(명사)
ko_txt

<Text: 미리 알 아고다 월 할인코드 이용 호텔 예약...>

In [75]:
words=['제주','제주도','강릉','남해','고성','속초','담양','태백','영월','태안']

text_lst=[]
text_lst=[txt for txt in ko_txt if txt in words]
cnt=Counter(text_lst)
print(cnt)
count_rank=cnt.most_common()
print(count_rank)
dic_txt=dict(count_rank)
print(dic_txt)
df2=pd.DataFrame(count_rank,columns=['지역명','빈도수'])
df2

Counter({'남해': 10, '속초': 10, '제주도': 6, '태안': 4, '강릉': 4, '고성': 4, '제주': 2})
[('남해', 10), ('속초', 10), ('제주도', 6), ('태안', 4), ('강릉', 4), ('고성', 4), ('제주', 2)]
{'남해': 10, '속초': 10, '제주도': 6, '태안': 4, '강릉': 4, '고성': 4, '제주': 2}


,지역명,빈도수
0,남해,10
1,속초,10
2,제주도,6
3,태안,4
4,강릉,4
5,고성,4
6,제주,2
